# 多模态嵌入

多模态嵌入的目的是将不同类型的数据（如图像和文本）映射到同一个共享的向量空间。



## CLIP

CLIP 采用双编码器架构，分别将图像和文本映射到同一个共享的向量空间中。

CLIP 在训练时采用了对比学习 (Contrastive Learning) 策略。

在处理一批图文数据时，模型的目标是最大化正确图文对的向量相似度，同时最小化所有错误配对的相似度。

通过这种“拉近正例，推远负例”的方式，模型从海量数据中学会了将语义相关的图像和文本在向量空间中拉近。

CLIP 将一个传统的分类任务，转化为一个“图文检索”问题——例如，要判断一张图片是不是猫，只需计算图片向量与“a photo of a cat”文本向量的相似度即可。

这使得 CLIP 无需针对特定任务进行微调，就能实现对视觉概念的泛化理解。

## 常用多模态嵌入模型

由北京智源人工智能研究院（BAAI）开发的 **BGE-M3**。

核心特性“M3”:

- 多语言性 (Multi-Linguality)：原生支持超过 100 种语言的文本与图像处理，能够轻松实现跨语言的图文检索。

- 多功能性 (Multi-Functionality)：在单一模型内同时支持密集检索（Dense Retrieval）、多向量检索（Multi-Vector Retrieval）和稀疏检索（Sparse Retrieval），为不同应用场景提供了灵活的检索策略。

- 多粒度性 (Multi-Granularity)：能够有效处理从短句到长达 8192 个 token 的长文档，覆盖了更广泛的应用需求。


它不同于 CLIP 对整张图进行编码的方式，而是采用网格嵌入 (Grid-Based Embeddings)，将图像分割为多个网格单元并独立编码。

这著提升了模型对图像局部细节的捕捉能力，在处理多物体重叠等复杂场景时更具优势。

In [4]:
import os
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"
import torch
from visual_bge.visual_bge.modeling import Visualized_BGE

model = Visualized_BGE(model_name_bge="BAAI/bge-base-en-v1.5",model_weight="models/bge/Visualized_base_en_v1.5.pth")
model.eval()

with torch.no_grad():
    text_emb = model.encode(text="datawhale开源组织的logo")
    img_emb_1 = model.encode(image="images/datawhale01.png")
    multi_emb_1 = model.encode(image="images/datawhale01.png", text="datawhale开源组织的logo")
    img_emb_2 = model.encode(image="images/datawhale02.png")
    multi_emb_2 = model.encode(image="images/datawhale02.png", text="datawhale开源组织的logo")

# 计算相似度
sim_1 = img_emb_1 @ img_emb_2.T
sim_2 = img_emb_1 @ multi_emb_1.T
sim_3 = text_emb @ multi_emb_1.T
sim_4 = multi_emb_1 @ multi_emb_2.T

print("=== 相似度计算结果 ===")
print(f"纯图像 vs 纯图像: {sim_1}")
print(f"图文结合1 vs 纯图像: {sim_2}")
print(f"图文结合1 vs 纯文本: {sim_3}")
print(f"图文结合1 vs 图文结合2: {sim_4}")


=== 相似度计算结果 ===
纯图像 vs 纯图像: tensor([[0.8318]])
图文结合1 vs 纯图像: tensor([[0.8291]])
图文结合1 vs 纯文本: tensor([[0.7627]])
图文结合1 vs 图文结合2: tensor([[0.9058]])
